In [8]:
# from datasets import load_market_data
import pandas as pd
# from modules._file_paths import OPTION_DATA
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta, TH
# from vasco.cc_financial_plots import general_plots
# from vasco import plot
import glob
import os
import timeit

from modules._utils import load_data
from modules import HistoricalData
from main import get_time_interval


In [12]:
dt1 = datetime(2023, 4, 20, 14, 45, 0)
dt2 = datetime(2023, 4, 20, 15, 30, 0)
delta = dt2 - dt1

days = delta.days

print(days)

0


In [3]:
tmp_dict = {"type":'ttt',
            "underlying":3,
            "expiry":2,
            "strike":'self.strike'
            }
str(tmp_dict)

"{'type': 'ttt', 'underlying': 3, 'expiry': 2, 'strike': 'self.strike'}"

In [14]:
dt_list = []
dt_list.append(dt1)
dt_list.append(dt2)

for i in range(len(dt_list)):
    print(dt_list[i])


2023-04-20 14:45:00
2023-04-20 15:30:00


In [7]:
dt = datetime
print(dt.strftime('%Y-%m-%d'))

TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'str' object

In [19]:
# def get_spot_v2(ctime:datetime):
#     grp_df = eis_data._data.loc[ctime].groupby(["Type","ExpiryDateTime", "Strike", "BidPrice", "AskPrice"], as_index=False).count()
#     ce_df = grp_df.loc[grp_df['Type']=='CE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]
#     pe_df = grp_df.loc[grp_df['Type']=='PE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]
#     merged_df = pd.merge(ce_df,pe_df,on=['ExpiryDateTime','Strike'],how='inner')

#     merged_df['spot_bid'] = merged_df['Strike'] + merged_df['BidPrice_x'] - merged_df['AskPrice_y']
#     merged_df['spot_ask'] = merged_df['Strike'] + merged_df['AskPrice_x'] - merged_df['BidPrice_y']

#     return (merged_df['spot_bid'].max() + merged_df['spot_ask'].min())/2


In [20]:
ctime = datetime.strptime('2020-01-01 09:16:00', "%Y-%m-%d %H:%M:%S")

In [24]:
eis_data = HistoricalData(source='eis_data', name = 'EIS DATA', underlying_instrument='BANKNIFTY',start_date='20200102', end_date='20200102', expiry_type='nearest_weekly')
eis_data.load_market_data()

In [25]:
# execution_time = timeit.timeit(lambda: eis_data.get_spot(ctime), number=10)
# # eis_data.get_spot('2020-01-01 09:16:00')
# print(eis_data.get_spot(ctime))
# print(execution_time)

In [44]:
from datetime import datetime,timedelta
from modules.global_variables import params
from modules._logger import logger,get_exception_line_no

def is_expiry_time(at_time_t:datetime, expiry, buffer) -> float: 
    '''
    Parameters
    ----------
    at_time_t: current timestamp
    expiry : expiry time of the option

    Returns
    -------
    True : if expiry time < buffer else False
    '''
    try:
        # calculates the time difference between expiry and present time
        # send in minutes
        # number_of_trading_mins_day = (6.25 * 60)


        # # caluclating the time left between two given days and converting the total time left into minutes
        # time_left, days_left = (pd.to_datetime(expiry) - pd.to_datetime(at_time_t)), (pd.to_datetime(expiry) - pd.to_datetime(at_time_t)).days
        # minutes_left = (time_left - timedelta(days=days_left)).seconds // 60
        # time_left_in_mins = (days_left * number_of_trading_mins_day) + minutes_left
        
        # if time_left_in_mins < buffer:
        #     return True
        # else:
        #     return False

        # separate logic
        if isinstance(at_time_t, np.datetime64):
            at_time_t = datetime.utcfromtimestamp(at_time_t.astype('O')/1e9)

        # check expiry datatype
        if isinstance(expiry, np.datetime64):
            expiry = datetime.utcfromtimestamp(expiry.astype('O')/1e9)

        time_delta = ((expiry - at_time_t).total_seconds())/60
        is_time = False
        # if time delta is less than unwind time
        unwind_time = params['UNWIND_TIME']
        print(f'time_delta={time_delta}, UNWIND_TIME={unwind_time}')
        if time_delta < params['UNWIND_TIME']:
            is_time = True

        return is_time
    
    except Exception as e:
        logger.warning(f'Error in is_expiry_time() in line : {get_exception_line_no()}, error : {e}')
        raise e


In [45]:
stime = datetime.strptime('2020-01-02 14:44:00', "%Y-%m-%d %H:%M:%S")
etime = datetime.strptime('2020-01-02 14:49:00', "%Y-%m-%d %H:%M:%S")

time_interval_list = get_time_interval(start_datetime=stime,
                                       end_datetime=etime,
                                       time_delta_type='minutes',
                                       time_delta_value=1)
# print(time_interval_list)
for t in time_interval_list:
    print(t)
    slice = eis_data.getSlice(t)
    print(slice)
    print(is_expiry_time(at_time_t=t,expiry=slice.get_slice_expiry(),buffer=0))

2020-01-02 14:44:00
time_delta=46.0, UNWIND_TIME=45
False
2020-01-02 14:45:00
time_delta=45.0, UNWIND_TIME=45
False
2020-01-02 14:46:00
time_delta=44.0, UNWIND_TIME=45
True
2020-01-02 14:47:00
time_delta=43.0, UNWIND_TIME=45
True
2020-01-02 14:48:00
time_delta=42.0, UNWIND_TIME=45
True
2020-01-02 14:49:00
time_delta=41.0, UNWIND_TIME=45
True


In [14]:
execution_time = timeit.timeit(lambda: eis_data.get_spot_v2(ctime), number=100)
# eis_data.get_spot('2020-01-01 09:16:00')
print(eis_data.get_spot_v2(ctime))
print(execution_time)

32334.75
4.469522411993239


### testing the unwind list on expiry date and time

In [2]:
stime = datetime.strptime('2020-01-01 09:16:00', "%Y-%m-%d %H:%M:%S")
etime = datetime.strptime('2020-01-01 10:16:00', "%Y-%m-%d %H:%M:%S")

time_interval_list = get_time_interval(start_datetime=stime,
                                       end_datetime=etime,
                                       time_delta_type='minutes',
                                       time_delta_value=1)

In [7]:
for t in time_interval_list:
    # v1 = eis_data.get_spot(t)
    v2 = eis_data.get_spot_v2(t)
    # print(f'v1={v1} and v2={v2}')
    # print(f'{(v1 == v2)}')
    print(f'{t},{round(v2,2)}')

2020-01-01 09:16:00,32334.75
2020-01-01 09:17:00,32323.72
2020-01-01 09:18:00,32324.9
2020-01-01 09:19:00,32339.72
2020-01-01 09:20:00,32339.45
2020-01-01 09:21:00,32336.72
2020-01-01 09:22:00,32329.15
2020-01-01 09:23:00,32340.82
2020-01-01 09:24:00,32345.8
2020-01-01 09:25:00,32340.32
2020-01-01 09:26:00,32346.28
2020-01-01 09:27:00,32352.02
2020-01-01 09:28:00,32347.7
2020-01-01 09:29:00,32350.38
2020-01-01 09:30:00,32360.12
2020-01-01 09:31:00,32362.32
2020-01-01 09:32:00,32357.2
2020-01-01 09:33:00,32354.15
2020-01-01 09:34:00,32348.48
2020-01-01 09:35:00,32344.88
2020-01-01 09:36:00,32333.42
2020-01-01 09:37:00,32329.98
2020-01-01 09:38:00,32325.7
2020-01-01 09:39:00,32329.05
2020-01-01 09:40:00,32328.52
2020-01-01 09:41:00,32317.85
2020-01-01 09:42:00,32298.6
2020-01-01 09:43:00,32297.4
2020-01-01 09:44:00,32284.82
2020-01-01 09:45:00,32290.68
2020-01-01 09:46:00,32249.7
2020-01-01 09:47:00,32226.28
2020-01-01 09:48:00,32227.38
2020-01-01 09:49:00,32247.05
2020-01-01 09:50:00,32

In [8]:
for t in time_interval_list:
    v1 = eis_data.get_spot(t)
    v2 = eis_data.get_spot_v2(t)
    print(f'v1={v1} and v2={v2}')
    print(f'{(v1 == v2)}')


v1=32334.75 and v2=32334.75
True
v1=32323.725 and v2=32323.725
True
v1=32324.9 and v2=32324.9
True
v1=32339.725 and v2=32339.725
True
v1=32339.45 and v2=32339.45
True
v1=32336.725000000002 and v2=32336.725000000002
True
v1=32329.15 and v2=32329.15
True
v1=32340.824999999997 and v2=32340.824999999997
True
v1=32345.8 and v2=32345.8
True
v1=32340.325 and v2=32340.325
True
v1=32346.275 and v2=32346.275
True
v1=32352.025 and v2=32352.025
True
v1=32347.7 and v2=32347.7
True
v1=32350.375 and v2=32350.375
True
v1=32360.125 and v2=32360.125
True
v1=32362.325 and v2=32362.325
True
v1=32357.199999999997 and v2=32357.199999999997
True
v1=32354.15 and v2=32354.15
True
v1=32348.475 and v2=32348.475
True
v1=32344.875 and v2=32344.875
True
v1=32333.425 and v2=32333.425
True
v1=32329.975 and v2=32329.975
True
v1=32325.699999999997 and v2=32325.699999999997
True
v1=32329.050000000003 and v2=32329.050000000003
True
v1=32328.525 and v2=32328.525
True
v1=32317.85 and v2=32317.85
True
v1=32298.6 and v2=3229

In [6]:
eis_data.get_spot(ctime)

32334.75

In [7]:
print(execution_time)

9.968576957006007


In [22]:
import time 

for i in range(100):
    start_time = time.time()

    eis_data.get_spot(ctime)

    end_time = time.time()

    execution_time = end_time - start_time

    print(f"Execution time: {execution_time} seconds")


Execution time: 0.5536301136016846 seconds
Execution time: 0.5316686630249023 seconds
Execution time: 0.4877917766571045 seconds
Execution time: 0.4562819004058838 seconds
Execution time: 0.5191583633422852 seconds
Execution time: 0.5058465003967285 seconds
Execution time: 0.4749422073364258 seconds
Execution time: 0.4811844825744629 seconds
Execution time: 0.5094938278198242 seconds
Execution time: 0.5160839557647705 seconds
Execution time: 0.4939389228820801 seconds
Execution time: 0.4746208190917969 seconds
Execution time: 0.47870707511901855 seconds
Execution time: 0.4713585376739502 seconds
Execution time: 0.46633291244506836 seconds
Execution time: 0.45832109451293945 seconds
Execution time: 0.47513532638549805 seconds
Execution time: 0.46647000312805176 seconds
Execution time: 0.5163142681121826 seconds
Execution time: 0.5274832248687744 seconds
Execution time: 0.5608630180358887 seconds
Execution time: 0.5533115863800049 seconds
Execution time: 0.5495665073394775 seconds
Execut

In [55]:
print(len(eis_data._data.loc[ctime].set_index(["ExpiryDateTime", "Strike"]).index))
print(len(eis_data._data.loc[ctime].set_index(["ExpiryDateTime", "Strike"]).index.unique()))

114
80


In [14]:
eis_data._data

,ExchToken,BidPrice,BidQty,AskPrice,AskQty,TTq,LTP,TotalTradedPrice,Instrument,ExpiryDate,ExpiryTime,Strike,Type,ExpiryDateTime
Date Time,,,,,,,,,,,,,,
2020-01-01 09:16:00,45716,4762.00,220.0,4913.10,220.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,27500.0,CE,2020-01-02 15:30:00
2020-01-01 09:17:00,45716,4768.10,220.0,4891.15,220.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,27500.0,CE,2020-01-02 15:30:00
2020-01-01 09:18:00,45716,4769.25,220.0,4891.45,220.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,27500.0,CE,2020-01-02 15:30:00
2020-01-01 09:19:00,45716,4779.05,380.0,4900.10,220.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,27500.0,CE,2020-01-02 15:30:00
2020-01-01 09:20:00,45716,4779.00,220.0,4899.15,380.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,27500.0,CE,2020-01-02 15:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 15:26:00,35073,3182.25,200.0,3292.90,300.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,35400.0,PE,2020-01-02 15:30:00
2020-01-01 15:27:00,35073,3181.30,200.0,3292.90,300.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,35400.0,PE,2020-01-02 15:30:00
2020-01-01 15:28:00,35073,3184.45,300.0,3292.90,300.0,0.0,0.0,0.0,BANKNIFTY,2020-01-02,15:30:00,35400.0,PE,2020-01-02 15:30:00


In [ ]:
grp

In [59]:
grp_df = eis_data._data.loc[ctime].groupby(["Type","ExpiryDateTime", "Strike", "BidPrice", "AskPrice"], as_index=False).count()

In [60]:
grp_df

,Type,ExpiryDateTime,Strike,BidPrice,AskPrice,ExchToken,BidQty,AskQty,TTq,LTP,TotalTradedPrice,Instrument,ExpiryDate,ExpiryTime
0,CE,2020-01-02 15:30:00,27500.0,4762.00,4913.10,1,1,1,1,1,1,1,1,1
1,CE,2020-01-02 15:30:00,27600.0,4143.75,4948.20,1,1,1,1,1,1,1,1,1
2,CE,2020-01-02 15:30:00,27700.0,3127.00,5257.15,1,1,1,1,1,1,1,1,1
3,CE,2020-01-02 15:30:00,27800.0,3027.00,5156.40,1,1,1,1,1,1,1,1,1
4,CE,2020-01-02 15:30:00,27900.0,2927.05,5612.25,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,PE,2020-01-02 15:30:00,35000.0,2585.25,2748.00,1,1,1,1,1,1,1,1,1
110,PE,2020-01-02 15:30:00,35100.0,2680.85,2951.05,1,1,1,1,1,1,1,1,1
111,PE,2020-01-02 15:30:00,35200.0,2780.65,3050.90,1,1,1,1,1,1,1,1,1
112,PE,2020-01-02 15:30:00,35300.0,2880.45,3112.45,1,1,1,1,1,1,1,1,1


In [61]:
# grp_df = eis_data._data.loc[ctime].groupby(["Type","ExpiryDateTime", "Strike", "BidPrice", "AskPrice"], as_index=False).count()
ce_df = grp_df.loc[grp_df['Type']=='CE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]
pe_df = grp_df.loc[grp_df['Type']=='PE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]

print(f'ce_df len={ce_df.shape[0]}')
print(f'pe_df len={pe_df.shape[0]}')

ce_df len=65
pe_df len=49


In [64]:
merged_df = pd.merge(ce_df,pe_df,on=['ExpiryDateTime','Strike'],how='inner')

merged_df

,Type_x,ExpiryDateTime,Strike,BidPrice_x,AskPrice_x,Type_y,BidPrice_y,AskPrice_y
0,CE,2020-01-02 15:30:00,30100.0,2046.85,2316.40,PE,0.55,2.90
1,CE,2020-01-02 15:30:00,30500.0,1811.90,1846.70,PE,0.60,1.95
2,CE,2020-01-02 15:30:00,30600.0,1674.70,1796.15,PE,0.55,2.00
3,CE,2020-01-02 15:30:00,30900.0,1374.70,1496.15,PE,1.20,1.70
4,CE,2020-01-02 15:30:00,31000.0,1312.95,1346.95,PE,1.80,1.85
5,CE,2020-01-02 15:30:00,31100.0,1213.05,1243.80,PE,1.90,2.00
6,CE,2020-01-02 15:30:00,31200.0,1113.55,1145.95,PE,1.90,2.00
7,CE,2020-01-02 15:30:00,31300.0,1013.50,1046.95,PE,1.85,2.00
8,CE,2020-01-02 15:30:00,31400.0,915.10,947.40,PE,2.45,2.55
9,CE,2020-01-02 15:30:00,31500.0,834.35,844.85,PE,3.30,3.35


In [66]:
merged_df['spot_bid'] = merged_df['Strike'] + merged_df['BidPrice_x'] - merged_df['AskPrice_y']
merged_df['spot_ask'] = merged_df['Strike'] + merged_df['AskPrice_x'] - merged_df['BidPrice_y']

In [67]:
merged_df

,Type_x,ExpiryDateTime,Strike,BidPrice_x,AskPrice_x,Type_y,BidPrice_y,AskPrice_y,spot_bid,spot_ask
0,CE,2020-01-02 15:30:00,30100.0,2046.85,2316.40,PE,0.55,2.90,32143.95,32415.85
1,CE,2020-01-02 15:30:00,30500.0,1811.90,1846.70,PE,0.60,1.95,32309.95,32346.10
2,CE,2020-01-02 15:30:00,30600.0,1674.70,1796.15,PE,0.55,2.00,32272.70,32395.60
3,CE,2020-01-02 15:30:00,30900.0,1374.70,1496.15,PE,1.20,1.70,32273.00,32394.95
4,CE,2020-01-02 15:30:00,31000.0,1312.95,1346.95,PE,1.80,1.85,32311.10,32345.15
5,CE,2020-01-02 15:30:00,31100.0,1213.05,1243.80,PE,1.90,2.00,32311.05,32341.90
6,CE,2020-01-02 15:30:00,31200.0,1113.55,1145.95,PE,1.90,2.00,32311.55,32344.05
7,CE,2020-01-02 15:30:00,31300.0,1013.50,1046.95,PE,1.85,2.00,32311.50,32345.10
8,CE,2020-01-02 15:30:00,31400.0,915.10,947.40,PE,2.45,2.55,32312.55,32344.95
9,CE,2020-01-02 15:30:00,31500.0,834.35,844.85,PE,3.30,3.35,32331.00,32341.55


In [69]:
def get_spot_v2(ctime:datetime):
    grp_df = eis_data._data.loc[ctime].groupby(["Type","ExpiryDateTime", "Strike", "BidPrice", "AskPrice"], as_index=False).count()
    ce_df = grp_df.loc[grp_df['Type']=='CE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]
    pe_df = grp_df.loc[grp_df['Type']=='PE',['Type','ExpiryDateTime','Strike','BidPrice','AskPrice']]
    merged_df = pd.merge(ce_df,pe_df,on=['ExpiryDateTime','Strike'],how='inner')

    merged_df['spot_bid'] = merged_df['Strike'] + merged_df['BidPrice_x'] - merged_df['AskPrice_y']
    merged_df['spot_ask'] = merged_df['Strike'] + merged_df['AskPrice_x'] - merged_df['BidPrice_y']

    (merged_df['spot_bid'].max() + merged_df['spot_ask'].min())/2

32334.75

In [17]:
from datetime import datetime

start_datetime='2021-03-10 09:16:00'
dt_obj = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
print(dt_obj.year,dt_obj.month,dt_obj.day)
# print(datetime(2021))
start_time = datetime(dt_obj.year, 
                      int(dt_obj.month),
                      int(dt_obj.day),
                      int(dt_obj.hour),
                      int(dt_obj.minute)) # Replace with your desired start time

print(start_time)

2021 3 10
2021-03-10 09:16:00


In [6]:
from datetime import datetime

start_datetime='2021-03-10 09:16:00'
end_datetime='09:20:00'
dt_obj = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
dt_only_time_1 = dt_obj.strftime('%H:%M:%S')
dt_only_time_2 = datetime.strptime(end_datetime, '%H:%M:%S')
print(dt_only_time_1)
print(dt_only_time_2)


ValueError: time data '2021-03-10 09:16:00' does not match format '%H:%M:%S'

In [5]:
def trading_time(t):
    return True

In [6]:
import numpy as np
import random 
# import Decimal

def get_spot(t):
    return random.choice(range(3577000,3617000))

for i in range(1,10):
    print(get_spot())

TypeError: get_spot() missing 1 required positional argument: 't'

In [ ]:
banknifty_20210310.loc[banknifty_20210310['StrikePrice'] >= 3578557,['StrikePrice']].apply(min)

StrikePrice    3580000
dtype: int64

In [9]:
def get_strike(spot):
    u = banknifty_20210310.loc[banknifty_20210310['StrikePrice'] > spot,['StrikePrice']].apply(min)[0]
    l = banknifty_20210310.loc[banknifty_20210310['StrikePrice'] < spot,['StrikePrice']].apply(max)[0]

    strike = 0
    if (u - spot*100) < (spot*100 - l):
        strike = u
    else:
        strike = l

    print(u,l)
    return strike

spot = get_spot(t)
print(spot)
print(get_strike(spot))

NameError: name 't' is not defined

In [8]:
def get_quote(t, q_type,source='hist'):
    spot = get_spot()
    strike = get_strike(spot)

    return strike

In [19]:
class Trade():
    def __init__(self, i, p, t, pos, trader, portfolio):
        self._instrument = i
        self._price = p
        self._time = t
        self._position = pos
        self._trader = trader
        self._portfolio = portfolio

    def execute(self):
        self._value = self._price * self._position
 
    def get_value(self):
        return self._value


In [20]:
trade_list = []
trade_list.append(Trade(1,2,3,4,5,6))

In [25]:
for t in trade_list:
    print(t._instrument)

1


In [ ]:
def generate_trade_strategy(t, percent):
    strike = get_quote(t)
    strike_above = get_quote(strike + round((strike * percent)/100,2))
    strike_below = get_quote(strike - round((strike * percent)/100,2))

    print(strike_below, strike, strike_above)

    
generate_trade_strategy()

In [27]:
class Portfolio():
    def update(self):
        pass


class Strategy():
    def __init__(self,portfolio,trade_interval,hedge_interval,unwind_time) -> None:
        self._trade_interval = trade_interval
        self._hedge_interval = hedge_interval
        self._unwind_time = unwind_time
        self._portfolio = portfolio

    def is_trading_time(self):
        return True

    def is_hedging_time(self):
        return True

    def is_unwind_time(self):
        return True

    def generate_unwind_strategy(self, t):
        pass

    def generate_trade_strategy(self, t, percent):
        trade_list = list()
        strike = get_quote(t)
        strike_above = get_quote(strike + round((strike * percent)/100,2))
        strike_below = get_quote(strike - round((strike * percent)/100,2))

        print(strike_below, strike, strike_above)
        return trade_list

    def generate_hedge_strategy(self, t):
        trade_list = list()
        # TODO: generate the list of trades
        return trade_list


class Blotter():
    def __init__(self,portfolio):
        self.data = pd.DataFrame(columns=['time','trade','trade_id'])
        self._portfolio = portfolio

    def add(self,trade_list):
        self.data.add(trade_list)
        self._portfolio.update(trade_list)


class Algo():
    # Assumptions:  
    #   1. The Blotter is attached to a the Algo
    #   2. The Portfolio is attached to the Blotter
    #   3. The Strategy is attached to the Algo
    #   4. A Portfolio is attached to a Strategy
    #   5. The Blotter and the Strategy share the same Portfolio

    def __init__(self, t, strategy, blotter):
        self._time = t
        self._strategy = strategy
        self._blotter = blotter

    def main(self):
        """
        """
        for t in self._time:
            if self._strategy.is_unwind_time(t):
                trade_list = self._strategy.generate_trade_strategy()
                if len(trade_list):
                    self._blotter.add(trade_list)
                else:
                    # TODO: need to verify this logic
                    print('trade skipped')
            elif self._strategy.is_trading_time(t):
                trade_list = self._strategy.generate_trade_strategy()
                self._blotter.add(trade_list)
            elif self._strategy.is_hedging_time(t):
                trade_list = self._strategy.generate_hedge_strategy()
                self._blotter.add(trade_list)
            else:
                # SKIP
                pass

            # TODO: add to BackTest


In [30]:
portfolio = Portfolio()
strategy = Strategy(portfolio,trade_interval=5,hedge_interval=2,unwind_time=60)
blotter = Blotter(portfolio)
algo = Algo(t==one_min_interval,strategy=strategy,blotter=blotter)

In [ ]:
from modules import Algo